In [ ]:
%%writefile a.cu

#include <opencv2/opencv.hpp>
#include <iostream>
#include <random>
#include <chrono>
#include <thread>
#include <vector>
#include <cuda_runtime.h>

using namespace std;

__global__ void func(unsigned char* img, unsigned char* new_img, int rescaleFactor){
  int n = 512;
  int n2 = 256;
  
  int x = threadIdx.x;
  int y = threadIdx.y;

  if(x<n&&y<n){
    int index = y*n+x;
    int index2 = ((y/n2)*n2)+(x/n2);
    new_img[index] = img[index2];
  }
}

int main()
{
    int num_threads = 4;
    cv::Size img_size(256, 256);
    int rescaleFactor = 2;

    cv::Mat img = cv::imread("/content/drive/MyDrive/images/img1.jpg", cv::IMREAD_GRAYSCALE);
    if(img.empty()){
      cout<<"empty"<<endl;
    }
    cv::Mat new_img(img_size.height * rescaleFactor, img_size.width * rescaleFactor, CV_8UC1);

    for (int i = 0; i < new_img.rows; i++) {
        for (int j = 0; j < new_img.cols; j++) {
            int x = i / rescaleFactor;
            int y = j / rescaleFactor;
            new_img.at<uchar>(i, j) = img.at<uchar>(x, y);
        }
    }

    

    bool success = cv::imwrite("output.jpg",new_img);

    uchar *dev_img;
    cudaMalloc((void**)&dev_img, img.total() * sizeof(uchar));
    cudaMemcpy(dev_img, img.data, img.total() * sizeof(uchar), cudaMemcpyHostToDevice);

    cv::Mat new_img_p(img_size.height * rescaleFactor, img_size.width * rescaleFactor, CV_8UC1);
    uchar *dev_new_img;
    cudaMalloc((void**)&dev_new_img, new_img_p.total() * sizeof(uchar));
    cudaMemcpy(dev_new_img, new_img_p.data, new_img_p.total() * sizeof(uchar), cudaMemcpyHostToDevice);

    int *dev_rf;
    cudaMalloc(&dev_rf, sizeof(int));
    cudaMemcpy(dev_rf, &rescaleFactor, sizeof(int), cudaMemcpyHostToDevice);

    dim3 blockDim(512,512);
    func<<<1,blockDim>>>(dev_img,dev_new_img,*dev_rf);

    cudaMemcpy(new_img_p.data, dev_new_img, new_img_p.total()*sizeof(uchar), cudaMemcpyDeviceToHost);

    cudaFree(dev_img);
    cudaFree(dev_new_img);
    cudaFree(dev_rf);

    success = cv::imwrite("output1.jpg",new_img_p);
    cout<<success<<endl;

    return 0;
}


Writing a.cu


In [ ]:
!nvcc -arch=sm_70 -I/usr/local/cuda/include -L/usr/local/cuda/lib64 -lcudart a.cu -o a `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(213): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(213): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::MultiBandBlender"

/usr/include/opencv4/opencv2/videostab/motion_stabilizing.hpp(106): warning #997-D: function "cv::videostab::IMotionStabilizer::stabilize(int, const std::vector<cv::

In [ ]:
!./a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
